# Installation and configuration

In [1]:
# pip install -r ./requirements.txt

In [2]:
# !pip install langchainhub

In [3]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

# LangChain

## LLM

In [4]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

In [5]:
llm = OpenAI(temperature=0.7)
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.run(question)


c:\Users\Kevin\Personal\Repositories\Python\Notebooks\LangChain\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n1. Justin Bieber was born on March 1, 1994.\n2. The Super Bowl is held in February of each year.\n3. Therefore, the Super Bowl in the year Justin Bieber was born would have been Super Bowl XXVIII, which took place on January 30, 1994.\n4. The winning team of Super Bowl XXVIII was the Dallas Cowboys, who defeated the Buffalo Bills by a score of 30-13.'

## CHAT

In [6]:
from langchain.callbacks import get_openai_callback
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [7]:
chat = ChatOpenAI(temperature=0)
messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    ),
]
with get_openai_callback() as cb:
    print(chat.invoke(messages))
    print(cb)

content="J'adore programmer." response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 34, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}
Tokens Used: 40
	Prompt Tokens: 34
	Completion Tokens: 6
Successful Requests: 1
Total Cost (USD): $6.300000000000001e-05


## Prompt templates

In [8]:
from langchain.prompts import PromptTemplate

In [9]:
template = """
your task is to translate the following text to {target_language}:

"{text}"
"""

prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['target_language', 'text'], template='\nyour task is to translate the following text to {target_language}:\n\n"{text}"\n')

In [10]:
llm = ChatOpenAI(temperature=0.7,model_name="gpt-3.5-turbo")
chain = LLMChain(prompt=prompt_template, llm=llm)
text = "I love programming a lot in my free time"
response = chain.invoke(input={"text": text, "target_language": "spanish"})
print(response)

{'text': '"Me encanta programar mucho en mi tiempo libre"', 'target_language': 'spanish'}


## Chains

In [11]:
from langchain.chains import SimpleSequentialChain

In [12]:
llm = ChatOpenAI(temperature=0.7,model_name="gpt-3.5-turbo")

In [13]:
template1 = """
Create an {amount} of items in a JSON list following the schema below, try to repeat at least 1 product name more than once.:
{{
    name: str,
    amount: int,
}}
"""
prompt_template1 = PromptTemplate.from_template(template1)
chain1 = LLMChain(prompt=prompt_template1, llm=llm)
response = chain1.invoke("4")
print(response["text"])


[
    {
        "name": "Apple",
        "amount": 5
    },
    {
        "name": "Banana",
        "amount": 3
    },
    {
        "name": "Orange",
        "amount": 2
    },
    {
        "name": "Apple",
        "amount": 7
    }
]


In [14]:
template2 = """from the JSON list {data}, get the mean, median, and mode of the data return it in a JSON object."""
prompt_template2 = PromptTemplate.from_template(template2)
chain2 = LLMChain(prompt=prompt_template2, llm=llm)

In [15]:
overall_chain = SimpleSequentialChain(chains=[chain1, chain2],verbose=True)
output = overall_chain.invoke("6")



> Entering new SimpleSequentialChain chain...
[
    {
        "name": "Apple",
        "amount": 5
    },
    {
        "name": "Banana",
        "amount": 3
    },
    {
        "name": "Orange",
        "amount": 2
    },
    {
        "name": "Grapes",
        "amount": 4
    },
    {
        "name": "Apple",
        "amount": 2
    },
    {
        "name": "Mango",
        "amount": 6
    }
]
{
    "mean": 3.6666666666666665,
    "median": 4,
    "mode": "Apple"
}

> Finished chain.


In [16]:
print(output["output"])

{
    "mean": 3.6666666666666665,
    "median": 4,
    "mode": "Apple"
}


## ReAct Agent (RAG)
allow your model to interact with real world data or external services, like databases or web APIs.

In [17]:
from langchain.schema import(
    SystemMessage,
    HumanMessage,
    AIMessage
)
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

In [18]:
python_repl = PythonREPL()
python_repl.run('print([n for n in range (1, 10) if n % 2 == 0])')


Python REPL can execute arbitrary code. Use with caution.


'[2, 4, 6, 8]\n'

In [19]:
llm = ChatOpenAI(temperature=0.7 , model_name="gpt-4-turbo-preview")

In [20]:
agent_executor = create_python_agent(llm=llm, tool=PythonREPLTool(),verbose=True)

In [21]:
prompt = 'Find the mean of the first 10 even numbers and print the result, force to display 3 decimals.'
response = agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To find the mean of the first 10 even numbers, I need to generate these numbers, calculate their sum, and then divide by the count. The first 10 even numbers start from 2 and end at 20 (2, 4, 6, 8, 10, 12, 14, 16, 18, 20). After calculating the mean, I will format the result to display with 3 decimal places.
Action: Python_REPL
Action Input: print(f"{sum(range(2, 21, 2)) / 10:.3f}")
Observation: 11.000

Thought:I now know the final answer
Final Answer: 11.000

> Finished chain.


In [22]:
response["output"]

'11.000'

## More advanced RAG

In [23]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

In [24]:
llm = ChatOpenAI(model_name="gpt-4-turbo-preview",temperature=0.7)

In [25]:
template = """
answer the following questions in spanish:
{question}
"""

In [26]:
prompt_template = PromptTemplate.from_template(template)

In [27]:
pip install langchainhub

In [28]:
#the way how langchain decides which agent to use
prompt = hub.pull("hwchase17/react")

### tools available for RAG

In [29]:
#1 python REPL tool
python_repl=PythonREPLTool()
python_repl_tool = Tool(
    name="PythonREPLTool",
    func=python_repl.run,
    description="Util when you need to run python code. You should input python code to use it."
)

#2 wikipedia query tool
api_wrapper = WikipediaAPIWrapper()
wikipedia= WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name="WikipediaTool",
    func=wikipedia.run,
    description="Util when you need to search information inside wikipedia."
)

#3 DuckDuckGo search tool
duckduckgo = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name="DuckDuckGoTool",
    func=duckduckgo.run,
    description="Util when you need to search information inside DuckDuckGo. You should use it when you need to find information across the web."
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

In [30]:
agent = create_react_agent(llm=llm, tools=tools,prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

In [31]:
question = "What is the capital of France?"
output= agent_executor.invoke({
    'input': prompt_template.format(question=question)
})
print(output["output"])



> Entering new AgentExecutor chain...
I need to find the capital of France and then translate the answer into Spanish.

Action: WikipediaTool

Action Input: Capital of France


KeyboardInterrupt: 

In [ ]:
question = "What does wikipedia say about France?, first search in english and then translate it into spanish."
output= agent_executor.invoke({
    'input': prompt_template.format(question=question)
})
print(output["output"])



> Entering new AgentExecutor chain...
I need to first find the information about France from Wikipedia in English and then translate that information into Spanish.

Action: WikipediaTool

Action Input: France
Page: France
Summary: France, officially the French Republic, is a country located primarily in Western Europe. It also includes overseas regions and territories in the Americas and the Atlantic, Pacific and Indian oceans, giving it one of the largest discontiguous exclusive economic zones in the world. Metropolitan France shares borders with Belgium and Luxembourg to the north, Germany to the north east, Switzerland to the east, Italy and Monaco to the south east, Andorra and Spain to the south, and a maritime border with the United Kingdom to the north west. Its metropolitan area extends from the Rhine to the Atlantic Ocean and from the Mediterranean Sea to the English Channel and the North Sea. Its overseas territories include French Guiana in South America, Saint Pierre and 

KeyboardInterrupt: 

In [ ]:
question = "search how to make a tamal in the web?"
output= agent_executor.invoke({
    'input': prompt_template.format(question=question)
})
print(output["output"])



> Entering new AgentExecutor chain...
To answer the question in Spanish, I need to search for a recipe or instructions on how to make a tamal using a web search tool.

Action: DuckDuckGoTool
Action Input: how to make a tamal recipe

KeyboardInterrupt: 

c:\Users\Kevin\Personal\Repositories\Python\Notebooks\LangChain\.venv\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [ ]:
question = "what is the square root of 7?"
output= agent_executor.invoke({
    'input': prompt_template.format(question=question)
})
print(output["output"])



> Entering new AgentExecutor chain...
To answer this question in Spanish, I need to calculate the square root of 7 first and then translate the answer into Spanish.

Action: PythonREPLTool
Action Input: import math
math.sqrt(7)Observation: 2.6457513110645907
Thought: Now that I know the square root of 7 is approximately 2.6457513110645907, I can translate this number into Spanish.
Final Answer: La raíz cuadrada de 7 es aproximadamente 2.6457513110645907.

> Finished chain.
La raíz cuadrada de 7 es aproximadamente 2.6457513110645907.


# Vector Stores (Pinecone)

## Vectors pinecone

In [32]:
load_dotenv(find_dotenv())

True

In [33]:
from pinecone import Pinecone

c:\Users\Kevin\Personal\Repositories\Python\Notebooks\LangChain\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [34]:
pc=Pinecone()

In [35]:
pc.list_indexes()

{'indexes': [{'dimension': 3072,
              'host': 'langchain-wjit0ko.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [36]:
index_name = "langchain"

In [37]:
import random

In [38]:
vectors = [[random.random() for _ in range(3072)] for _ in range(5)]

In [39]:
ids = list('abcde')

In [40]:
index = pc.Index(index_name)

In [41]:
index.upsert(vectors=zip(ids,vectors))

{'upserted_count': 5}

### Index Query


In [42]:
query_vector = [random.random() for _ in range(3072)]

In [43]:
index.query(vector=query_vector, top_k=3,include_values=False)

{'matches': [{'id': 'a', 'score': 0.756501675, 'values': []},
             {'id': 'c', 'score': 0.753943, 'values': []},
             {'id': 'b', 'score': 0.752138615, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

### Namespaces

In [44]:
index = pc.Index(index_name)

In [45]:
vectors = [[random.random() for _ in range(3072)] for _ in range(2)]
ids = list('fg')
index.upsert(vectors=zip(ids,vectors),namespace="test1")

vectors = [[random.random() for _ in range(3072)] for _ in range(2)]
ids = list('fg')
index.upsert(vectors=zip(ids,vectors),namespace="test2")

{'upserted_count': 2}

In [46]:
index.describe_index_stats()

{'dimension': 3072,
 'index_fullness': 9e-05,
 'namespaces': {'': {'vector_count': 5},
                'test1': {'vector_count': 2},
                'test2': {'vector_count': 2}},
 'total_vector_count': 9}

In [47]:
index.fetch(ids=['f'],namespace="test1")

{'namespace': 'test1',
 'usage': {'read_units': 1},
 'vectors': {'f': {'id': 'f',
                   'values': [0.105392426,
                              0.705357552,
                              0.896330416,
                              0.325474709,
                              0.998529732,
                              0.0105705848,
                              0.892372,
                              0.609721065,
                              0.0323307179,
                              0.54340297,
                              0.875545382,
                              0.428965181,
                              0.522227526,
                              0.403141975,
                              0.872400522,
                              0.307764083,
                              0.816251576,
                              0.922824,
                              0.823024809,
                              0.0831430256,
                              0.321263611,
                   

### Vectors with LLM

In [48]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [51]:
with open('ElAlquimista.txt', encoding='utf-8') as file:
    text = file.read()

In [52]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)

In [74]:
chunks=text_splitter.create_documents([text])
print(chunks[0].page_content)

Paulo Coelho
El Alquimista






Para J.
Alquimista que conoce y utiliza los secretos de la Gran Obra


## Embedding


In [60]:
from langchain_openai import OpenAIEmbeddings

In [81]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large",dimensions=3072)

In [82]:
vector = embeddings.embed_query(chunks[0].page_content)
vector

[0.0059898342152981225,
 -0.023945277616664568,
 -0.0002943947398584535,
 0.017547684331866476,
 -0.007663051020849278,
 0.009308146543377072,
 -0.009083176279190195,
 -0.005863288441693004,
 -0.020219205287763144,
 0.03577027945629359,
 -0.04803115885447842,
 0.019867690646954907,
 -0.003095099236627437,
 -0.001461428247262336,
 0.014890221689175228,
 0.0476937034581981,
 0.001994853921923175,
 -1.581822533861837e-05,
 -0.031270874172556036,
 -0.0455283637340769,
 -0.034195487606985445,
 -0.004949346743433813,
 0.005462560391440754,
 -0.01570573889685266,
 0.021259693690949957,
 -0.007965354580519769,
 0.015340162217548983,
 0.025618493490893218,
 -0.007937233297496408,
 0.012324154380463028,
 0.005469590479365968,
 -0.008738690329323413,
 0.027882257239934927,
 -0.05911094622832715,
 -0.03017414134067749,
 -0.01829289879764676,
 -0.03211451080061181,
 0.018433504281441054,
 -0.02083787537692207,
 -0.024634247653753123,
 -0.012774094908836783,
 -0.04012907739359184,
 -0.03391427291410

## Pinecone and embeddings

In [83]:
import os
import pinecone
from langchain_community.vectorstores import Pinecone
from pinecone import PodSpec

In [84]:
index_name = "el-alquimista"
pc=pinecone.Pinecone()

In [73]:
# create_index
# pc.create_index(name=index_name, dimension=3072,metric="cosine",spec=PodSpec(environment="gcp-starter"))

In [85]:
vector_store = Pinecone.from_documents(chunks,embeddings,index_name=index_name)

In [86]:
index=pc.Index(index_name)
index.describe_index_stats()

{'dimension': 3072,
 'index_fullness': 0.00603,
 'namespaces': {'': {'vector_count': 603}},
 'total_vector_count': 603}

## using Pinecone with LLM

In [88]:
question = "¿Qué papel juega el encuentro de Santiago con el rey de Salem en su viaje hacia la realización de su Leyenda Personal enel libro?"

In [91]:
result = vector_store.similarity_search(question)
for r in result:
    print(r.page_content)
    print("\n")

"Soy el rey de Salem", había dicho el viejo.
-¿Por qué un rey conversa con un pastor? -preguntó el muchacho, avergonzado y admiradísimo.
-Existen varias razones. Pero la más importante es que tú has sido capaz de cumplir tu Leyenda Personal.
El muchacho no sabía qué era eso de la Leyenda Personal.


"Sin embargo, en vez de encontrar a un hombre santo, nuestro héroe entró en una sala y vio una actividad inmensa; mercaderes que entraban y salían, personas conversando en los rincones, una pequeña orquesta que tocaba melodías suaves y una mesa repleta de los más deliciosos manjares de aquella región del mundo. El sabio conversaba con todos, y el joven tuvo que esperar dos horas para que le atendiera.


Cuando salió del bar estaba muy contento. Se había acordado de que uno de los proveedores del Mercader traía los cristales en caravanas que cruzaban el desierto. Mantuvo a Urim y Tumim en las manos; gracias a aquellas dos piedras había reemprendido el camino hacia su tesoro.
"Siempre estoy c

In [92]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

In [93]:
llm = ChatOpenAI(temperature=0.7,model_name="gpt-4-turbo-preview")

In [94]:
retriever = vector_store.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [95]:
chain = RetrievalQA.from_chain_type(llm=llm,retriever=retriever,chain_type='stuff')

In [96]:
question = "¿Qué papel juega el encuentro de Santiago con el rey de Salem en su viaje hacia la realización de su Leyenda Personal enel libro?"
aswer = chain.invoke(question)
print(aswer)

{'query': '¿Qué papel juega el encuentro de Santiago con el rey de Salem en su viaje hacia la realización de su Leyenda Personal enel libro?', 'result': 'El encuentro de Santiago con el rey de Salem, Melquisedec, juega un papel crucial en su viaje hacia la realización de su Leyenda Personal en "El Alquimista" de Paulo Coelho. Este encuentro marca un punto de inflexión en la vida de Santiago, proporcionándole la orientación, el conocimiento y la motivación necesarios para perseguir su verdadero destino. Aquí se destacan varios aspectos fundamentales de su influencia en el viaje de Santiago:\n\n1. **Introducción al concepto de Leyenda Personal**: Melquisedec introduce a Santiago al concepto de Leyenda Personal, que es el destino o la vida que uno está destinado a vivir. Esta noción inspira a Santiago a seguir sus sueños y a buscar su tesoro personal, que cree está cerca de las Pirámides de Egipto.\n\n2. **Conciencia de las señales**: El rey de Salem enseña a Santiago a entender y a segui

In [98]:
print(aswer["result"])

El encuentro de Santiago con el rey de Salem, Melquisedec, juega un papel crucial en su viaje hacia la realización de su Leyenda Personal en "El Alquimista" de Paulo Coelho. Este encuentro marca un punto de inflexión en la vida de Santiago, proporcionándole la orientación, el conocimiento y la motivación necesarios para perseguir su verdadero destino. Aquí se destacan varios aspectos fundamentales de su influencia en el viaje de Santiago:

1. **Introducción al concepto de Leyenda Personal**: Melquisedec introduce a Santiago al concepto de Leyenda Personal, que es el destino o la vida que uno está destinado a vivir. Esta noción inspira a Santiago a seguir sus sueños y a buscar su tesoro personal, que cree está cerca de las Pirámides de Egipto.

2. **Conciencia de las señales**: El rey de Salem enseña a Santiago a entender y a seguir las señales del mundo, que son indicaciones del alma del mundo sobre cómo uno puede cumplir su Leyenda Personal. Este entendimiento es crucial para la trave